In [13]:
SIZE=49
MIN_COUNT=5
WINDOW=6
NEGATIVE_WORDS=6
EPOCH_SIZE=4

In [24]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# shuffle
from random import shuffle

# logging
import logging
import os.path
import sys

program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))

2017-04-13 18:11:10,132 : INFO : running /Users/SamBerndt/miniconda3/envs/deeplearning/lib/python3.5/site-packages/ipykernel_launcher.py -f /Users/SamBerndt/Library/Jupyter/runtime/kernel-8205026b-1632-4cf3-9431-39b7289a700e.json


In [25]:


class LabeledLineSentence(object):

    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(
                        utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        shuffle(self.sentences)
        return self.sentences
sources = {'test-neg.txt':'TEST_NEG', 'test-pos.txt':'TEST_POS', 'train-neg.txt':'TRAIN_NEG', 'train-pos.txt':'TRAIN_POS'} #'train-unsup.txt':'TRAIN_UNS'}

sentences = LabeledLineSentence(sources)


model = Doc2Vec(min_count=MIN_COUNT, window=WINDOW, size=SIZE, sample=1e-4, negative=5, workers=7)

model.build_vocab(sentences.to_array())





2017-04-13 18:11:11,764 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2017-04-13 18:11:11,775 : INFO : collecting all words and their counts
2017-04-13 18:11:11,777 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-13 18:11:11,788 : INFO : collected 2319 word types and 454 unique tags from a corpus of 454 examples and 36514 words
2017-04-13 18:11:11,789 : INFO : Loading a fresh vocabulary
2017-04-13 18:11:11,795 : INFO : min_count=5 retains 848 unique words (36% of original 2319, drops 1471)
2017-04-13 18:11:11,797 : INFO : min_count=5 leaves 33645 word corpus (92% of original 36514, drops 2869)
2017-04-13 18:11:11,801 : INFO : deleting the raw counts dictionary of 2319 items
2017-04-13 18:11:11,804 : INFO : sample=0.0001 downsamples 554 most-common words
2017-04-13 18:11:11,805 : INFO : downsampling leaves estimated 9926 word corpus (29.5% of prior 33645)
2017-04-13 18:11:11,807 : INFO : estimated required memo

In [27]:
sentences.sentences_perm()

[LabeledSentence(words=['given', 'a', 'complete', 'binary', 'tree', 'count', 'the', 'number', 'of', 'nodes', 'definition', 'of', 'a', 'complete', 'binary', 'tree', 'from', 'wikipedia', 'in', 'a', 'complete', 'binary', 'tree', 'every', 'level', 'except', 'possibly', 'the', 'last', 'is', 'completely', 'filled', 'and', 'all', 'nodes', 'in', 'the', 'last', 'level', 'are', 'as', 'far', 'left', 'as', 'possible', 'it', 'can', 'have', 'between', 'and', 'h', 'nodes', 'inclusive', 'at', 'the', 'last', 'level', 'h'], tags=['TRAIN_NEG_101']),
 LabeledSentence(words=['given', 'two', 'binary', 'strings', 'return', 'their', 'sum', 'also', 'a', 'binary', 'string', 'for', 'example', 'a', 'b', 'return'], tags=['TRAIN_POS_207']),
 LabeledSentence(words=['sort', 'a', 'linked', 'list', 'in', 'o', 'n', 'log', 'n', 'time', 'using', 'constant', 'space', 'complexity'], tags=['TRAIN_NEG_135']),
 LabeledSentence(words=['given', 'a', 'non', 'negative', 'integer', 'represented', 'as', 'a', 'non', 'empty', 'array',

In [29]:
for epoch in range(EPOCH_SIZE):
    logger.info('Epoch %d' % epoch)
    model.train(sentences.sentences_perm(),  total_examples=len(sentences.sentences_perm()))

2017-04-13 18:14:19,032 : INFO : Epoch 0


NameError: name 'train_model' is not defined

In [ ]:
model.save('./problems.d2v')


In [6]:
model = Doc2Vec.load('./problems.d2v')

2017-04-13 18:05:58,707 : INFO : loading Doc2Vec object from ./problems.d2v
2017-04-13 18:05:58,759 : INFO : loading docvecs recursively from ./problems.d2v.docvecs.* with mmap=None
2017-04-13 18:05:58,760 : INFO : loading wv recursively from ./problems.d2v.wv.* with mmap=None
2017-04-13 18:05:58,762 : INFO : setting ignored attribute syn0norm to None
2017-04-13 18:05:58,763 : INFO : setting ignored attribute cum_table to None
2017-04-13 18:05:58,765 : INFO : loaded ./problems.d2v


In [9]:
model.most_similar('police')

[('tonight', 0.9893763661384583),
 ('rob', 0.9885302186012268),
 ('broken', 0.9876682758331299),
 ('night', 0.9874764084815979),
 ('alerting', 0.9806720614433289),
 ('money', 0.9758768081665039),
 ('contact', 0.9733415842056274),
 ('thief', 0.9689029455184937),
 ('automatically', 0.9660723209381104),
 ('amount', 0.963667631149292)]

In [10]:
model.docvecs['TRAIN_POS_0']

array([ -1.52504981e-01,  -2.57091910e-01,  -3.52481633e-01,
         3.13576199e-02,  -5.41601554e-02,  -1.97111517e-01,
        -2.27686018e-01,  -1.37802765e-01,  -2.11380180e-02,
        -1.12766288e-01,  -2.97669899e-02,   1.59161896e-01,
         2.16760695e-01,  -1.84225792e-06,   9.74352136e-02,
        -5.45294303e-03,  -3.31546925e-03,  -2.41953153e-02,
         7.68047571e-02,  -4.79129292e-02,   2.56738812e-01,
        -4.56496954e-01,  -3.83166447e-02,   2.19659925e-01,
         8.69752690e-02,  -1.37020126e-01,  -1.08903266e-01,
         2.01000780e-01,   1.40617177e-01,  -1.80494040e-01,
        -4.01686251e-01,  -2.45995238e-01,  -1.63651809e-01,
        -2.11079955e-01,   8.13655257e-02,  -2.85958759e-02,
        -9.76955611e-03,   1.86850950e-01,  -1.06689095e-01,
        -1.74664244e-01,   2.12197840e-01,  -1.61512867e-01,
        -2.17965432e-02,  -3.41028005e-01,   5.63425384e-02,
         1.10611115e-02,   2.38345489e-02,  -1.31562114e-01,
        -3.35772812e-01]

In [11]:
model.docvecs['TEST_POS_0']

array([ 0.03355458, -0.21483544, -0.25010738,  0.0985546 , -0.14522858,
       -0.11774437, -0.04604067,  0.06599008,  0.06029277,  0.0994892 ,
       -0.32745373, -0.1261864 ,  0.16569152,  0.00845462,  0.10075148,
       -0.075981  ,  0.14648494, -0.01804082, -0.09041696, -0.1054535 ,
        0.12392563, -0.22225887, -0.13533619,  0.34095198,  0.2041374 ,
       -0.208387  , -0.14299341,  0.37939337, -0.19009849,  0.01004872,
       -0.06837716, -0.04650459,  0.13240223, -0.03121288, -0.20930262,
       -0.23346426, -0.117372  ,  0.20317326, -0.1379028 , -0.17463209,
        0.06350625, -0.36917263,  0.03329552, -0.19660477, -0.02340458,
       -0.15864563, -0.09952213, -0.18488741, -0.29679909], dtype=float32)

In [18]:
logger.info('Sentiment')
train_arrays = numpy.zeros((210+154, SIZE))
train_labels = numpy.zeros(210+154)

for i in range(210):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_labels[i] = 1
for i in range(154):
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[210 + i] = model.docvecs[prefix_train_neg]
    train_labels[210 + i] = 0

2017-04-13 18:07:48,386 : INFO : Sentiment


In [20]:
test_arrays = numpy.zeros((52+38, SIZE))
test_labels = numpy.zeros(52+38)

for i in range(52):
    prefix_test_pos = 'TEST_POS_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_labels[i] = 1
for i in range(38):
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[52 + i] = model.docvecs[prefix_test_neg]
    test_labels[52 + i] = 0


In [21]:
from sklearn.linear_model import LogisticRegression

logger.info('Fitting')
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

print(classifier.score(test_arrays, test_labels))


2017-04-13 18:07:56,178 : INFO : Fitting


0.655555555556


In [22]:
from sklearn import svm
classifier = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
classifier.fit(train_arrays, train_labels)
print(classifier.score(test_arrays,test_labels))

0.6


In [23]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, AveragePooling1D
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from keras.utils import np_utils
import numpy as np
batch_size = 20
num_classes = 2
epochs = 15

# input image dimensions
img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)
x_train=train_arrays.reshape(train_arrays.shape[0], SIZE,1)
x_test=test_arrays.reshape(test_arrays.shape[0], SIZE,1)
input_shape = (SIZE)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#print('x_train shape:', x_train.shape)
#print(x_train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
y_train=np_utils.to_categorical(train_labels, 2)
y_test=np_utils.to_categorical(test_labels, 2)
#print(y_test)



num_classes=2
sequence_input = (SIZE,1)
Kmodel = Sequential()
Kmodel.add(Conv1D(64, kernel_size=1,activation='linear',input_shape=sequence_input))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(AveragePooling1D(3))
#Kmodel.add(Conv1D(128, 2, activation='linear'))
#Kmodel.add(LeakyReLU(alpha=.001)) 
#Kmodel.add(AveragePooling1D())
#print(Kmodel.output_shape)
Kmodel.add(Flatten())
#print(Kmodel.output_shape)
Kmodel.add(Dense(128, activation='linear'))
Kmodel.add(LeakyReLU(alpha=.001))
#print(Kmodel.output_shape)
Kmodel.add(Dense(num_classes, activation='softmax'))
#print(Kmodel.output_shape)
Kmodel.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
Kmodel.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1)
score = Kmodel.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
Kmodel.metrics_names

Using TensorFlow backend.


Epoch 1/15
364/364 [==============================] - 0s - loss: 0.6906 - acc: 0.5742     
Epoch 2/15
364/364 [==============================] - 0s - loss: 0.6857 - acc: 0.5769     
Epoch 3/15
364/364 [==============================] - 0s - loss: 0.6841 - acc: 0.5769     
Epoch 4/15
364/364 [==============================] - 0s - loss: 0.6849 - acc: 0.5769     
Epoch 5/15
364/364 [==============================] - 0s - loss: 0.6839 - acc: 0.5769     
Epoch 6/15
364/364 [==============================] - 0s - loss: 0.6852 - acc: 0.5769     
Epoch 7/15
364/364 [==============================] - 0s - loss: 0.6855 - acc: 0.5769     
Epoch 8/15
364/364 [==============================] - 0s - loss: 0.6824 - acc: 0.5769     
Epoch 9/15
364/364 [==============================] - 0s - loss: 0.6850 - acc: 0.5769     
Epoch 10/15
364/364 [==============================] - 0s - loss: 0.6847 - acc: 0.5769     
Epoch 11/15
364/364 [==============================] - 0s - loss: 0.6828 - acc: 0.5769   

['loss', 'acc']

import numpy as np
np.random.seed(123)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, normalization
from keras.utils import np_utils
from keras.datasets import mnist, cifar10
from keras.layers.advanced_activations import LeakyReLU
from sklearn.decomposition import PCA


# Basic Prepocess function for MNIST
# Uses PCA and Normalization
def pca_mnist(X, X_test):
    X = X.astype('float32') / 255.
    X_test = X_test.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    X_test = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=100)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X_test = pca.inverse_transform(pca.transform(X_test))
    X = X.reshape(X.shape[0], 1, 28, 28)
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
    print(X.shape)
    return X, X_test

#Preprocess for cifar10
#Doesn't really affect performance
def pca_cifar_pre(X):
    X2 = np.transpose(X, (3, 0, 1, 2))
    temp1 = 0
    temp2 = 0
    temp3 = 0
    for i in range(3): #individual PCA for each color 
        temp_X = X2[i][:][:][:]
        temp_X = np.transpose(temp_X, (0, 2, 1))
        temp_X = pca_cifar(temp_X)
        if i == 0:
            temp1 = temp_X
        elif i == 1:
            temp2 = temp_X
        else:
            temp3 = temp_X
    X_temp = np.array([temp1, temp2, temp3])
    # print(X.shape)
    X = np.transpose(X, (1, 2, 3, 0))
    X = X.astype('float32') / 255.
    X = X.reshape(X.shape[0], 3, 32, 32)
    return X


def pca_cifar(X):
    X = X.astype('float32') / 255.
    X = X.reshape((len(X), np.prod(X.shape[1:])))
    print(X.shape)
    pca = PCA(n_components=1024)
    pca.fit(X)
    X = pca.inverse_transform(pca.transform(X))
    X = X.reshape(X.shape[0], 32, 32)
    print(X.shape)
    return X


(X_train, y_train), (X_test, y_test) = mnist.load_data()



# 6. Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

# 7. Define model architecture
model = Sequential()

model.add(Convolution(64, kernel_size=(5, 5), data_format="channels_first", activation='relu',input_shape=(1, 28, 28)))


print(model.output_shape)
model.add(Convolution2D(64,kernel_size=(5, 5), activation='linear')) #Convolutinal Leaky Layer
model.add(LeakyReLU(alpha=.001))
print(model.output_shape)
model.add(MaxPooling2D(pool_size=(2, 2))) #Vanila Pooling
print(model.output_shape)
model.add(Dropout(0.25))

model.add(Flatten())
print(model.output_shape)
model.add(Dense(128, activation='relu'))
print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


model.summary

#Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
#Fit model on training data
model.fit(X_train, Y_train, batch_size=32, epochs=20, verbose=1)


#Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=1)
print(score)
